In [1]:
import astropy.units as u
import matplotlib as plt
import numpy as np
import time
from numba import jit
#import scipy.interpolate as interpl
from numba.extending import overload

from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import


import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [ ]:
def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)

In [ ]:
def trap_geo_para(x, y, z, RFAMP, DCAMP, MIRROR):
    #need to account for hole in the bottom for needle? how much space for laser slits?
    RFamp = [0, 0, 0, 0, 0];
    Vtopcap = DCAMP;
    Vside = RFAMP;
    Vbottom = RFAMP;
    Mirror = [MIRROR, MIRROR, MIRROR, MIRROR, MIRROR];
    V0 = np.zeros((np.size(x), np.size(y), np.size(z)));
    V0RF = np.zeros((np.size(x), np.size(y), np.size(z)));
    Vneedle =  0;
    #Needle is zero, endcap has DC voltage, parabolic mirror has RF
    #How to define the actual geometery
    #various constants for accurate geometery
    Hmirror = 1; #mirror height
    Htopcap = 1; #distance from bottom to top end cap
    Ropening = 1; # radius at Hmirror
    Hneedle1 = - 1; #height of needle inside trap up until the beginning of the needle tip
    Rneedle = 1; #radius of needle
    #SlopeNeedle = 1; #slope of needle from base to tip **Maybe important later, for now just call needle cylinder**
    lattice_points = np.empty((9999999,3));
    marker = 1; 
    mirror_equation = (Hmirror / Ropening**2) * (x**2 +y**2); #the equation describing the shape of the mirror (paraboloid). form of z=(1/a^2)(x^2+y^2)
    
    #boundry conditions (depend on those constants one needs to define.)
    for i in range(len(x)):
        for j in range(len(y)):
            for k in range(len(z)):
    
#def trap_geo(x,y,z,RFAMP,DCAMP,RINGV):
    
#    RFamp = [0,0,0,0,0,0,0,0];
#    Vt = RFAMP;
#    Vb = RFAMP;
#    V0 = np.zeros((np.size(x),np.size(y),np.size(z))); #initialize the matrix for electric potential
#    V0dc= np.zeros((np.size(x),np.size(y),np.size(z)));
#    Vend=RINGV+DCAMP;
#    Ring=[RINGV,RINGV,RINGV,RINGV,RINGV,RINGV,RINGV,RINGV];
#    Vtdc=Vend;
#    Vbdc=Vend;
     #for the main RF ring:
#    RF_R = .015; #max radius in units m
#    RF_r = .0024; #min radius in units m
#    rcone=.00519; #radius where inner cut begins
#    slope=np.tan(79*np.pi/180)**(-1);
#    const=slope-rcone;
#    H=.001; #height of electrode above z axis at the start of the cut
#    c3=H-slope*rcone;
    
#    h = .000551; #displacement of cone ends from center
#    R2=.00111/2;#radius of big cone at displacement h
#    R1=.00108/2;#radius of little cone at displacement h from center
#    h1=.00295;#total height of exposed cone from tip to the flat plate section 
#    c1=h-np.sqrt(3)*R1;#small cone axial offset from center
#    c2=h-R2;
#    lattice_points=np.empty((9999999,3));
#    marker=1;
    for i in range(len(x)):
        for j in range(len(y)):
            for k in range(len(z)):
                if ((z[k] > h) and (z[k]<(h+h1)) and (z[k] > np.sqrt(x[i]**2 + y[j]**2)+c2) and (z[k] < (np.sqrt(3)*np.sqrt(x[i]**2 + y[j]**2)+c1))):

                   # disp(['hi'])
                    V0[i,j,k] = Vt;
                    V0dc[i]=Vtdc;
                    V0dc[j]=Vtdc;
                    V0dc[k]=Vtdc;

                elif((z[k]>h+h1) and (z[k]<(h+h1+.001)) and (z[k]<(np.sqrt(3)*np.sqrt(x[i]**2 + y[j]**2)+c1))):     #and (abs(x[i])<.00675/2)
                  #  disp(['hi'])
                    V0[i,j,k] = Vt;
                    V0dc[i,j,k]=Vtdc;
                    #then the voltage plates on the bottom:
                elif ((z[k] < -h) and (z[k]>-h-h1) and (z[k] < -(np.sqrt(x[i]**2 + y[j]**2)+c2)) and (z[k] > -(np.sqrt(3)*np.sqrt(x[i]**2 + y[j]**2)+c1))):
                    V0[i,j,k] = Vb;
                    V0dc[i,j,k]=Vbdc;
                elif((z[k]<-(h+h1)) and (z[k]>-(h+h1+.001)) and (z[k]>-(np.sqrt(3)*np.sqrt(x[i]**2 + y[j]**2)+c1) )):
                    V0[i,j,k] = Vb;
                    V0dc[i,j,k]=Vbdc;
                elif ((x[i]**2+y[j]**2>RF_r**2) and (z[k]<.001) and (z[k]>-.001) and (abs(z[k])< slope*(np.sqrt(x[i]**2+y[j]**2))+ c3)):
                    angle=abs(np.arctan(y[j]/x[i]))*180/np.pi;#####calculate the angle in polar coordinates, convert to degrees

                    if  ((x[i]>0) and (y[j]>0) and (12.5<angle) and (angle<32.5)):
                        V0[i,j,k] = RFamp[0];
                        V0dc[i,j,k]=Ring[0];

                    elif ((x[i]>0) and (y[j]>0) and (57.5<angle) and (angle<77.5)):
                        V0[i,j,k] = RFamp[1];
                        V0dc[i,j,k]=Ring[1];
                    elif ((x[i]<0) and (y[j]>0) and (102.5<180-angle) and (180-angle<122.5)):
                        V0[i,j,k] = RFamp[2]
                        V0dc[i,j,k]=Ring[2];

                    elif ((x[i]<0) and (y[j]>0) and (147.5<180-angle) and (180-angle<167.5)):
                        V0[i,j,k] = RFamp[3];
                        V0dc[i,j,k]=Ring[3];

                    elif ((x[i]<0) and (y[j]<0) and (192.5<180+angle) and (180+angle<212.5)):
                        V0[i,j,k] = RFamp[4]

                        V0dc[i,j,k]=Ring[4];

                    elif ((x[i]<0) and (y[j]<0) and (237.5<180+angle) and (180+angle<257.5)):
                        V0[i,j,k] = RFamp[5]

                        V0dc[i,j,k]=Ring[5];

                    elif ((x[i]>0) and (y[j]<0) and (282.5<360-angle) and (360-angle<302.5)):
                        V0[i,j,k] = RFamp[6];

                        V0dc[i,j,k]=Ring[6];

                    elif (x[i]>0 and y[j]<0 and (327.5<360-angle) and (360-angle<347.5)):
                        V0[i,j,k] = RFamp[7]
                        V0dc[i,j,k]=Ring[7];

                    else: ###else it is a point that must get updated
                        V0[i,j,k] = 0;
                        V0dc[i,j,k]=0;
                        lattice_points[marker]=[i,j,k];
                        marker=marker+1;
                    #break

                elif ((x[i]**2+y[j]**2) < (RF_r)**2 and abs(z[k])<h):
                    
                    lattice_points[marker]=[i,j,k];
                    marker=marker+1;
                else:
                    lattice_points[marker]=[i,j,k];
                    marker=marker+1;
    return(lattice_points, marker, V0,V0dc)

In [ ]:
#This function initializes all of the trap parameters and calculates the
#potential using the finite difference method
#for the BIv00 trap:
#@overload(spl)
#@jit(nopython=True)



    # Initialize trap parameters
    
    #for the main RF ring:
#     RF_R = .015; #max radius in units m
#     RF_r = .0024; #min radius in units m
#     rcone=.00519; #radius where inner cut begins
#     slope=np.tan(79*np.pi/180)**(-1);
#     const=slope-rcone;
#     H=.001; #height of electrode above z axis at the start of the cut
#     c3=H-slope*rcone;

    #for the voltage bias plates

#     h = .000551; #displacement of cone ends from center
#     R2=.00111/2;#radius of big cone at displacement h
#     R1=.00108/2;#radius of little cone at displacement h from center
#     h1=.00295;#total height of exposed cone from tip to the flat plate section 
#     c1=h-np.sqrt(3)*R1;#small cone axial offset from center
#     c2=h-R2; # big cone axial offset from center
    # Voltage signals RF
#     #AMP=900;
#     RFamp = [0,0,0,0,0,0,0,0]; #V, amplitude of applied RF signal
#     Vt = RFAMP;
#     Vb = RFAMP;
    #VOLTAGE DC
#     Vend=RINGV+DCAMP;
#     Ring=[RINGV,RINGV,RINGV,RINGV,RINGV,RINGV,RINGV,RINGV];
#     Vtdc=Vend;
#     Vbdc=Vend;

    # Grid properties
gmax = .005; #max and min grid point position
#number of gridpoints in position space
points=100;
#####number of steps (i.e. 99 steps=100 grid poingts)
steps = points-1;
####initialize spatial coords
x = np.linspace(-gmax,gmax,num=steps); #m, sets boundaries of solution space
y = np.linspace(-gmax,gmax,num=steps); #m, sets boundaries of solution space
z = np.linspace(-gmax,gmax,num=steps); #m, sets boundaries of solution space
#####record middle positions
xmid = int((np.size(x))/2);
ymid = int((np.size(y))/2);
zmid = int((np.size(z))/2);
#####initialize the lattice points, that is the points that will be
#####updated in the simulation
lattice_points=np.empty((9999999,3));
marker=1;
V0 = np.zeros((np.size(x),np.size(y),np.size(z))); #initialize the matrix for electric potential
V0dc= np.zeros((np.size(x),np.size(y),np.size(z)));
V0_temp=np.zeros((np.size(x),np.size(y),np.size(z)));
V0dc_temp=np.zeros((np.size(x),np.size(y),np.size(z)));
its = 250; #number of iterations
CenterV = np.zeros((its,1)); #keep track of the potential value at the center at each iteration

#lattice_points,marker,V0,V0dc=trap_geo(x,y,z,RFAMP,DCAMP,RINGV)
lattice_points,marker,V0,V0dc=trap_geo(x,y,z,800,5,0)
#V0_temp=V0    
#V0dc_temp=V0dc;

 #  disp(marker)###display the number of lattice points. mostly used for
  #  debugging initially
    #-------------------------------------------------------------------------
#
    ## Define the initial potential using FDM first in big steps
    #disp(['Calculating V in big steps...'])
def FDM_BIv03(lattice_points,marker,V0,V0dc,points):
    
    for iterate in range(1,10): 
        for I in range (1,marker):
            point=lattice_points[I];
            a=int(point[0]);
            b=int(point[1]);
            c=int(point[2]);
            if ((a>3 and b>3 and c>3 and a<points-5 and b<points-5 and c<points-5)):
                V0_temp[a,b,c] = 1/6*(V0[a+4,b,c]+V0[a-4,b,c]+V0[a,b+4,c]+V0[a,b-4,c]+V0[a,b,c+4]+V0[a,b,c-4]);
                V0dc_temp[a,b,c] = 1/6*(V0dc[a+4,b,c]+V0dc[a-4,b,c]+V0dc[a,b+4,c]+V0dc[a,b-4,c]+V0dc[a,b,c+4]+V0dc[a,b,c-4]);
            I+=1
       
        
        CenterV[iterate] = V0[xmid,ymid,zmid];
        V0=V0_temp;
        V0dc=V0dc_temp;
        iterate+=1;

    
#     ## Define the initial potential using FDM in small steps
#    # disp(['Calculating V in small steps...'])


    for iterate in range(91,100):
        for I in range (1,marker):
            point=lattice_points[I];
            a=int(point[0]);
            b=int(point[1]);
            c=int(point[2]);
            if ((a>2 and b>2 and c>2 and a<points-4 and b<points-4 and c<points-4)):
                V0_temp[a,b,c] = 1/6*(V0[a+2,b,c]+V0[a-2,b,c]+V0[a,b+2,c]+V0[a,b-2,c]+V0[a,b,c+2]+V0[a,b,c-2]);
                V0dc_temp[a,b,c] = 1/6*(V0dc[a+2,b,c]+V0dc[a-2,b,c]+V0dc[a,b+2,c]+V0dc[a,b-2,c]+V0dc[a,b,c+2]+V0dc[a,b,c-2]);
            I+=1
                    
                   
                   
        CenterV[iterate] = (V0[xmid,ymid,zmid]+V0[xmid-2,ymid,zmid]+V0[xmid+2,ymid,zmid]+V0[xmid,ymid-2,zmid]+V0[xmid,ymid+2,zmid]+V0[xmid,ymid,zmid-2]+V0[xmid,ymid,zmid+2])/7;
        V0=V0_temp;
        V0dc=V0dc_temp;
    
    
#    # disp(['Calculating V in baby steps...'])
    for iterate in range (100,130):
        for I in range (1,marker):
            point=lattice_points[I];
            a=int(point[0]);
            b=int(point[1]);
            c=int(point[2]);
            if ((a>1 and b>1 and c>1 and a<points-3 and b<points-3 and c<points-3)):
                
                V0_temp[a,b,c] = 1/6*(V0[a+1,b,c]+V0[a-1,b,c]+V0[a,b+1,c]+V0[a,b-1,c]+V0[a,b,c+1]+V0[a,b,c-1]);
                V0dc_temp[a,b,c] = 1/6*(V0dc[a+1,b,c]+V0dc[a-1,b,c]+V0dc[a,b+1,c]+V0dc[a,b-1,c]+V0dc[a,b,c+1]+V0dc[a,b,c-1]);
            I+=1
            
            
        CenterV[iterate] = (V0[xmid,ymid,zmid]+V0[xmid-1,ymid,zmid]+ V0[xmid+1,ymid,zmid]+V0[xmid,ymid-1,zmid]+ V0[xmid,ymid+1,zmid]+V0[xmid,ymid,zmid-1]+ V0[xmid,ymid,zmid+1])/7;
        V0=V0_temp;
        V0dc=V0dc_temp;
        iterate+=1
#}
    ## Test convergence
    
    #figure; plot(CenterV(2,(iterate-1)));
    #CenterV[its]; xlabel('Number of Iterations'); ylabel('Voltage at trap center'); title('Monitoring the convergence of the finite difference method')
    
    if (CenterV[np.size(its)] - CenterV[np.size(its-1)] > 1):
        print(['Convergence test FAIL'])
    else:
        print(['Convergence test PASS'])
    
   
    ## Trim and Interpolate!This makes things look much cleaner

    #Vrf= interpl.Rbf(V0,function="linear";
    #Vdc= interpl.Rbf(V0dc,function="linear");
    Vrf=V0
    Vdc=V0dc
    #RegularGrid
    #steps = steps*4;

    
    
    
    return Vrf,Vdc,x,y,z,xmid,ymid,zmid,gmax,steps

In [ ]:
def plt_pots(x,y,z,xmid,ymid,zmid, Vrf,Vdc):
    fig = plt.figure()
    ax1 = fig.add_subplot(111, projection='3d')
    
    surf1=ax1.plot_surface(x,y, Vdc[:,:,zmid], cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    fig.colorbar(surf1)
    plt.show()
    
    fig = plt.figure()
    ax1rf = fig.add_subplot(111, projection='3d')
    
    surf1rf=ax1rf.plot_surface(x,y, Vrf[:,:,zmid], cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    fig.colorbar(surf1)
    plt.show()
    
    fig = plt.figure()
    ax2 = fig.add_subplot(111, projection='3d')
    surf2=ax2.plot_surface(x, z,Vdc[:,ymid,:], cmap=cm.coolwarm,
                             linewidth=0, antialiased=False)
    fig.colorbar(surf2)
    plt.show()
    
    fig = plt.figure()
    ax2rf = fig.add_subplot(111, projection='3d')
    surf2rf=ax2rf.plot_surface(x, z,Vrf[:,ymid,:], cmap=cm.coolwarm,
                             linewidth=0, antialiased=False)
    fig.colorbar(surf2)
    plt.show()
    
    fig = plt.figure()
    ax3 = fig.add_subplot(111, projection='3d')
    surf3=ax3.plot_surface(y,z,Vdc[xmid,:,:], cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    fig.colorbar(surf3)
    plt.show()
    
    fig = plt.figure()
    ax3rf = fig.add_subplot(111, projection='3d')
    surf3rf=ax3rf.plot_surface(y,z,Vrf[xmid,:,:], cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    fig.colorbar(surf3)
    plt.show()
    
    fig.colorbar(surf1)
    fig.colorbar(surf1rf)
    fig.colorbar(surf2)
    fig.colorbar(surf2rf)
    fig.colorbar(surf3)
    fig.colorbar(surf3rf)
    return

In [ ]:
plt.imshow(FDM_BIv03(lattice_points,marker,V0,V0dc,points)[1][:,:,50])
